# 1D Stationary

In [3]:
using gmsh

using Gridap, GridapGmsh
using Gridap.Fields, Gridap.Geometry

## Generate Mesh

In [4]:
T0 = 20.0;
P  = 10;
L  = 5;

lc1 = 0.01;

In [5]:
gmsh.initialize()

gmsh.model.add("bar_type3")
geo  = gmsh.model.geo;
mesh = gmsh.model.mesh;

# Points
geo.addPoint(0, 0, 0, lc1, 1)
geo.addPoint(L, 0, 0, lc1, 2)

# Lines
geo.addLine(1, 2, 1)

geo.synchronize()

# Physical domains
geo.addPhysicalGroup(0, [1], 1)
geo.addPhysicalGroup(0, [2], 2)
gmsh.model.setPhysicalName(0, 1, "bnd_left")
gmsh.model.setPhysicalName(0, 2, "bnd_right")

geo.addPhysicalGroup(1, [1], 1)
gmsh.model.setPhysicalName(1, 1, "domain")

geo.synchronize()

# Generate mesh
mesh.generate(1);
gmsh.write("geo/bar_type3.msh")

Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Done meshing 1D (Wall 0s, CPU 0s)
Info    : 501 nodes 502 elements
Info    : Writing 'geo/bar_type3.msh'...
Info    : Done writing 'geo/bar_type3.msh'


In [6]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.9.4
License       : GNU General Public License
Build OS      : Windows64-sdk
Build date    : 20220203
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen NoSocklenT ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.15.0 (real arithmtic)
OCC version   : 7.6.0
MED version   : 4.1.0
Packaged by   : nt authority system
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


## Gridap

In [7]:
# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)

# Load mesh
model = GmshDiscreteModel("geo/bar_type3.msh")
Ω = Triangulation(model)
dΩ = Measure(Ω, 2*order)
Γ = BoundaryTriangulation(model, tags = ["bnd_left"])
dΓ = Measure(Γ, 2*order)

# Construct Lagrangian test space
V = TestFESpace(model, reffe, dirichlet_tags = ["bnd_right"])
U = TrialFESpace(V, [T0])

Info    : Reading 'geo/bar_type3.msh'...
Info    : 3 entities
Info    : 501 nodes
Info    : 502 elements
Info    : Done reading 'geo/bar_type3.msh'


TrialFESpace()

In [8]:
a(u,v) = ∫( ∇(u) ⋅ ∇(v) )dΩ
b(v)   = ∫( 0 * v )dΩ + ∫( P * v )dΓ

op = AffineFEOperator(a, b, U, V)

# Solve the linear FE system with LU solver
ls = LUSolver()
solver = LinearFESolver(ls)
uh = solve(solver, op);

Qh = -∇(uh);

In [9]:
writevtk(Ω,"sol/sol_1d_stationary",cellfields=["Th"=>uh,"Qh"=>Qh])

(["sol/sol_1d_stationary.vtu"],)

# 1D Transient

## Gridap

In [10]:
# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)

# Load mesh
model = GmshDiscreteModel("geo/bar_type3.msh")
Ω = Triangulation(model)
dΩ = Measure(Ω, 2*order)
Γ = BoundaryTriangulation(model, tags = ["bnd_left"])
dΓ = Measure(Γ, 2*order)

# Construct Lagrangian test space
V = TestFESpace(model, reffe, dirichlet_tags = ["bnd_right"])

# Build a transient trial space with Dirichlet conditions
g(x, t::Real) = T0 * (x[1] == L);
g(t::Real) = x -> g(x,t)
U  = TransientTrialFESpace(V, g)

Pf(t) = P * (t > 1);

Info    : Reading 'geo/bar_type3.msh'...
Info    : 3 entities
Info    : 501 nodes
Info    : 502 elements
Info    : Done reading 'geo/bar_type3.msh'


In [11]:
m(t,u,v) = ∫( v * u )dΩ
a(t,u,v) = ∫( ∇(u) ⋅ ∇(v) )dΩ
b(t,v) = ∫( 0 * v )dΩ + ∫( Pf(t) * v )dΓ

op = TransientAffineFEOperator(m, a, b, U, V)

TransientFEOperatorFromWeakForm()

In [12]:
solver = LUSolver()

Δt = 0.25;
θ = 1
ode_solver = ThetaMethod(solver, Δt, θ)

u0 = interpolate_everywhere(T0, U(0.0))
t0 = 0.0
T = 50;

uht = solve(ode_solver, op, u0, t0, T)

Gridap.ODEs.TransientFETools.TransientFESolution(GenericODESolution(), TransientTrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Nothing}, TrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Nothing}}}(UnconstrainedFESpace(), g, TrialFESpace()))

In [13]:
createpvd("sol/sol_1d_transient") do pvd
    for (uh,t) in uht
        Qh = -∇(uh);
        
        pvd[t] = createvtk(Ω, "sol/sol_1d_transient_$t" * ".vtu", cellfields=["Th" => uh, "Qh" => Qh])
    end
end

201-element Vector{String}:
 "sol/sol_1d_transient.pvd"
 "sol/sol_1d_transient_0.25.vtu"
 "sol/sol_1d_transient_0.5.vtu"
 "sol/sol_1d_transient_0.75.vtu"
 "sol/sol_1d_transient_1.0.vtu"
 "sol/sol_1d_transient_1.25.vtu"
 "sol/sol_1d_transient_1.5.vtu"
 "sol/sol_1d_transient_1.75.vtu"
 "sol/sol_1d_transient_2.0.vtu"
 "sol/sol_1d_transient_2.25.vtu"
 "sol/sol_1d_transient_2.5.vtu"
 "sol/sol_1d_transient_2.75.vtu"
 "sol/sol_1d_transient_3.0.vtu"
 ⋮
 "sol/sol_1d_transient_47.25.vtu"
 "sol/sol_1d_transient_47.5.vtu"
 "sol/sol_1d_transient_47.75.vtu"
 "sol/sol_1d_transient_48.0.vtu"
 "sol/sol_1d_transient_48.25.vtu"
 "sol/sol_1d_transient_48.5.vtu"
 "sol/sol_1d_transient_48.75.vtu"
 "sol/sol_1d_transient_49.0.vtu"
 "sol/sol_1d_transient_49.25.vtu"
 "sol/sol_1d_transient_49.5.vtu"
 "sol/sol_1d_transient_49.75.vtu"
 "sol/sol_1d_transient_50.0.vtu"

# 2D Transient

In [136]:
# Define Lagrangian reference element
order = 2;
reffe = ReferenceFE(lagrangian, Float64, order)

# Load mesh
model = GmshDiscreteModel("geo/coupled_inductor.msh")
Ω = Triangulation(model)
dΩ = Measure(Ω, 2*order)
#Γ = BoundaryTriangulation(model, tags = ["bnd_left"])
#dΓ = Measure(Γ, 2*order)

# Construct Lagrangian test space
V = TestFESpace(model, reffe, dirichlet_tags = ["dirichlet"])

# Build a transient trial space with Dirichlet conditions
g(x, t::Real) = 0;
g(t::Real) = x -> g(x,t)
U  = TransientTrialFESpace(V, g)

Info    : Reading 'geo/coupled_inductor.msh'...
Info    : 47 entities
Info    : 6957 nodes
Info    : 13912 elements
Info    : Done reading 'geo/coupled_inductor.msh'


TransientTrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Nothing}, TrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Nothing}}}(UnconstrainedFESpace(), g, TrialFESpace())

In [159]:
labels = get_face_labeling(model)
dimension = num_cell_dims(model)
tags = get_face_tag(labels, dimension)

const tag_air = get_tag_from_name(labels, "Air")
const tag_core = get_tag_from_name(labels, "Core")
const tag_wdg_left = get_tag_from_name(labels, "Winding Left")
const tag_wdg_right = get_tag_from_name(labels, "Winding Right")

function rcp(tag)
    if(tag == tag_air)
        return 1.225 * 700;
    elseif(tag == tag_core)
        return 5.0e3 * 750;
    elseif(tag == tag_wdg_left || tag == tag_wdg_right)
        return 8.96e3 * 1.46e3;
    else
        error("rho * cp not defined for tag {$tag}")
        return -1;
    end
end

function k(tag)
    if(tag == tag_air)
        return 0.025;
    elseif(tag == tag_core)
        return 4;
    elseif(tag == tag_wdg_left || tag == tag_wdg_right)
        return 398;
    else
        error("k not defined for tag {$tag}")
        return -1;
    end
end

function Psource(tag)
    if(tag == tag_air)
        return 0.0;
    elseif(tag == tag_core)
        return 50.0 / 190e-6;
    elseif(tag == tag_wdg_left || tag == tag_wdg_right)
        return 50.0 / 60e-6;
    else
        error("Psource not defined for tag {$tag}")
        return -1;
    end
end

Pf(t) = 0;

τ = CellField(tags, Ω);

In [160]:
m(t,u,v) = ∫( (rcp ∘ τ) * v * u )dΩ
a(t,u,v) = ∫( (k ∘ τ) * ∇(u) ⋅ ∇(v) )dΩ
b(t,v) = ∫( (Psource ∘ τ) * v )dΩ; #+ ∫( Pf(t) * v )dΓ

op = TransientAffineFEOperator(m, a, b, U, V)

TransientFEOperatorFromWeakForm()

In [161]:
solver = LUSolver()

Δt = 2;
θ = 0.75
ode_solver = ThetaMethod(solver, Δt, θ)

u0 = interpolate_everywhere(0, U(0.0))
t0 = 0.0
T = 200;

uht = solve(ode_solver, op, u0, t0, T)

Gridap.ODEs.TransientFETools.TransientFESolution(GenericODESolution(), TransientTrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Nothing}, TrialFESpace{Gridap.FESpaces.UnconstrainedFESpace{Vector{Float64}, Nothing}}}(UnconstrainedFESpace(), g, TrialFESpace()))

In [162]:
createpvd("sol/sol_2d_transient") do pvd
    for (uh,t) in uht
        Qh = -∇(uh);
        
        pvd[t] = createvtk(Ω, "sol/sol_2d_transient_$t" * ".vtu", cellfields=["Th" => uh, "Qh" => Qh])
    end
end

101-element Vector{String}:
 "sol/sol_2d_transient.pvd"
 "sol/sol_2d_transient_2.0.vtu"
 "sol/sol_2d_transient_4.0.vtu"
 "sol/sol_2d_transient_6.0.vtu"
 "sol/sol_2d_transient_8.0.vtu"
 "sol/sol_2d_transient_10.0.vtu"
 "sol/sol_2d_transient_12.0.vtu"
 "sol/sol_2d_transient_14.0.vtu"
 "sol/sol_2d_transient_16.0.vtu"
 "sol/sol_2d_transient_18.0.vtu"
 "sol/sol_2d_transient_20.0.vtu"
 "sol/sol_2d_transient_22.0.vtu"
 "sol/sol_2d_transient_24.0.vtu"
 ⋮
 "sol/sol_2d_transient_178.0.vtu"
 "sol/sol_2d_transient_180.0.vtu"
 "sol/sol_2d_transient_182.0.vtu"
 "sol/sol_2d_transient_184.0.vtu"
 "sol/sol_2d_transient_186.0.vtu"
 "sol/sol_2d_transient_188.0.vtu"
 "sol/sol_2d_transient_190.0.vtu"
 "sol/sol_2d_transient_192.0.vtu"
 "sol/sol_2d_transient_194.0.vtu"
 "sol/sol_2d_transient_196.0.vtu"
 "sol/sol_2d_transient_198.0.vtu"
 "sol/sol_2d_transient_200.0.vtu"